In [7]:
import numpy as np
import faiss
import pickle
import os

from tensorflow.keras.models import load_model
lstm_model=load_model("C:/Users/DHARMESH M/Documents/Projects/SIG-COG/ML/link_state_lstm_model.h5")


def l2_normalize(vec):
    norm = np.linalg.norm(vec)
    return vec / norm if norm > 0 else vec

def create_sequence_embedding(window):
    return window.flatten().astype('float32')


state_dim=15*7
index = faiss.IndexFlatIP(state_dim)
self_rag_memory = []

def add_to_memory(sequence_window, action, outcome):
    embedding = create_sequence_embedding(sequence_window)
    embedding = l2_normalize(embedding)  
    index.add(np.array([embedding]))
    self_rag_memory.append((embedding, action, outcome))

def search_memory(sequence_window, top_k=1):
    embedding = create_sequence_embedding(sequence_window)
    embedding = l2_normalize(embedding)  
    if len(self_rag_memory) == 0:
        return None, None
    D, I = index.search(np.array([embedding]), top_k)
    return self_rag_memory[I[0][0]], D[0][0]
ACTIONS = [
    "Keep current settings",
    "Switch to 16QAM + Strong FEC",
    "Lower Modulation to QPSK + Max FEC",
    "Increase Tx Power",
    "Change Channel"
]

In [19]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.models import load_model

# -----------------------------
# 1. Load dataset and preprocess
# -----------------------------
df = pd.read_csv(r"C:\Users\DHARMESH M\Documents\Projects\SIG-COG\Dataset\features_dataset.csv")

features = ['SNR_dB','BER','PacketLoss_pct','Jitter_ms','Throughput_Mbps','SNR_trend','Jitter_spike']
X_raw = df[features].values
y_label = df['Label'].values

# Standardize features
scaler = StandardScaler()
X = scaler.fit_transform(X_raw)

# Encode labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y_label)
class_order = list(label_encoder.classes_)
print("Class Order:", class_order)

# -----------------------------
# 2. Create windowed sequences
# -----------------------------
window_size = 15
X_seq, y_seq = [], []
for i in range(len(X) - window_size + 1):
    X_seq.append(X[i:i + window_size])
    y_seq.append(y[i + window_size - 1])

X_seq = np.array(X_seq)
y_seq = np.array(y_seq)
print("Shape of X_seq:", X_seq.shape)
print("Shape of y_seq:", y_seq.shape)

# -----------------------------
# 3. Load your trained LSTM model
# -----------------------------
model_path = r"C:\Users\DHARMESH M\Documents\Projects\SIG-COG\ML\link_state_lstm.h5"  # <-- change if needed
model = load_model(model_path)

# -----------------------------
# 4. Pick balanced samples for simulation
# -----------------------------
np.random.seed(42)  # for reproducibility

# Get indices for each class
indices_per_class = {cls: np.where(y_seq == idx)[0] for idx, cls in enumerate(class_order)}

# Choose 5 random windows per class for simulation
samples_per_class = 5
selected_indices = []
for cls in class_order:
    idxs = indices_per_class[cls]
    if len(idxs) > 0:
        chosen = np.random.choice(idxs, size=min(samples_per_class, len(idxs)), replace=False)
        selected_indices.extend(chosen)

# Shuffle to simulate a realistic sequence
np.random.shuffle(selected_indices)

# -----------------------------
# 5. Simulation Loop
# -----------------------------
for step, idx in enumerate(selected_indices, start=1):
    window = X_seq[idx:idx+1]  # shape (1, 15, features)
    probs = model.predict(window, verbose=0)
    pred_class_idx = np.argmax(probs)
    pred_class = class_order[pred_class_idx]
    true_class = class_order[y_seq[idx]]

    print(f"\n[Step {step}] Window True Label: {true_class}")
    print(f"Window Prediction Probabilities: {probs}")
    print(f"Predicted State: {pred_class}")

    if pred_class == "Good":
        print("✅ Window classified as GOOD → No action needed.")
    else:
        print("⚠️ Window classified as", pred_class, "→ Action required!")


Class Order: ['Good', 'Moderate', 'Poor']
Shape of X_seq: (14986, 15, 7)
Shape of y_seq: (14986,)

[Step 1] Window True Label: Good
Window Prediction Probabilities: [[9.4825530e-01 5.1550705e-02 1.9394481e-04]]
Predicted State: Good
✅ Window classified as GOOD → No action needed.

[Step 2] Window True Label: Moderate
Window Prediction Probabilities: [[2.8539702e-04 9.9943703e-01 2.7749376e-04]]
Predicted State: Moderate
⚠️ Window classified as Moderate → Action required!

[Step 3] Window True Label: Good
Window Prediction Probabilities: [[9.9970645e-01 2.8618981e-04 7.3974179e-06]]
Predicted State: Good
✅ Window classified as GOOD → No action needed.

[Step 4] Window True Label: Moderate
Window Prediction Probabilities: [[3.3827734e-04 9.9936920e-01 2.9256818e-04]]
Predicted State: Moderate
⚠️ Window classified as Moderate → Action required!

[Step 5] Window True Label: Poor
Window Prediction Probabilities: [[1.4675227e-06 2.0887320e-04 9.9978966e-01]]
Predicted State: Poor
⚠️ Window c

In [20]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.models import load_model
import faiss
import random

# -----------------------------
# 1. Load dataset and preprocess
# -----------------------------
df = pd.read_csv(r"C:\Users\DHARMESH M\Documents\Projects\SIG-COG\Dataset\features_dataset.csv")

features = ['SNR_dB','BER','PacketLoss_pct','Jitter_ms','Throughput_Mbps','SNR_trend','Jitter_spike']
X_raw = df[features].values
y_label = df['Label'].values

# Standardize features
scaler = StandardScaler()
X = scaler.fit_transform(X_raw)

# Encode labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y_label)
class_order = list(label_encoder.classes_)
print("Class Order:", class_order)

# -----------------------------
# 2. Create windowed sequences
# -----------------------------
window_size = 15
X_seq, y_seq = [], []
for i in range(len(X) - window_size + 1):
    X_seq.append(X[i:i + window_size])
    y_seq.append(y[i + window_size - 1])

X_seq = np.array(X_seq)
y_seq = np.array(y_seq)
print("Shape of X_seq:", X_seq.shape)
print("Shape of y_seq:", y_seq.shape)

# -----------------------------
# 3. Load trained LSTM model
# -----------------------------
model_path = r"C:\Users\DHARMESH M\Documents\Projects\SIG-COG\ML\link_state_lstm.h5"
model = load_model(model_path)

# -----------------------------
# 4. Self-RAG Memory Setup
# -----------------------------
ACTIONS = [
    "Keep current settings",
    "Switch to 16QAM + Strong FEC",
    "Lower Modulation to QPSK + Max FEC",
    "Increase Tx Power",
    "Change Channel"
]

def l2_normalize(vec):
    norm = np.linalg.norm(vec)
    return vec / norm if norm > 0 else vec

def create_sequence_embedding(window):
    return window.flatten().astype('float32')

state_dim = window_size * len(features)
index = faiss.IndexFlatIP(state_dim)  # Inner product for cosine similarity (normalized)
self_rag_memory = []  # Stores tuples: (embedding, action, outcome)

def add_to_memory(sequence_window, action, outcome):
    embedding = l2_normalize(create_sequence_embedding(sequence_window))
    index.add(np.array([embedding]))
    self_rag_memory.append((embedding, action, outcome))

def search_memory(sequence_window, top_k=1):
    if len(self_rag_memory) == 0:
        return None, None
    embedding = l2_normalize(create_sequence_embedding(sequence_window))
    D, I = index.search(np.array([embedding]), top_k)
    return self_rag_memory[I[0][0]], D[0][0]

# -----------------------------
# 5. Pick balanced samples for simulation
# -----------------------------
np.random.seed(42)
indices_per_class = {cls: np.where(y_seq == idx)[0] for idx, cls in enumerate(class_order)}

samples_per_class = 5
selected_indices = []
for cls in class_order:
    idxs = indices_per_class[cls]
    if len(idxs) > 0:
        chosen = np.random.choice(idxs, size=min(samples_per_class, len(idxs)), replace=False)
        selected_indices.extend(chosen)

np.random.shuffle(selected_indices)

# -----------------------------
# 6. Simulation Loop with Self-RAG
# -----------------------------
for step, idx in enumerate(selected_indices, start=1):
    window = X_seq[idx]  # shape (15, 7)
    input_seq = np.expand_dims(window, axis=0)  # (1, 15, 7)

    probs = model.predict(input_seq, verbose=0)
    pred_class_idx = np.argmax(probs)
    pred_class = class_order[pred_class_idx]
    true_class = class_order[y_seq[idx]]

    print(f"\n[Step {step}] True Label: {true_class}")
    print(f"Predicted Probabilities: {probs}")
    print(f"Predicted State: {pred_class}")

    if pred_class == "Good":
        print("✅ Window classified as GOOD → No action needed.")
        continue

    # 🔹 Check Self-RAG memory for similar past cases
    matched_item, similarity = search_memory(window)
    if matched_item:
        prev_action, prev_outcome = matched_item[1], matched_item[2]
        print(f"🔍 Found similar case (sim={similarity:.2f}) → Previous Outcome: {prev_outcome}")
        action = prev_action
    else:
        print("ℹ️ No similar case in memory → Trying actions from scratch.")
        action = random.choice(ACTIONS)

    # 🔹 Simulate action effect
    new_window = window.copy()
    snr_boost = np.random.uniform(1.0, 3.0) if pred_class == "Poor" else np.random.uniform(0.5, 2.0)
    new_window[-1, 0] += snr_boost  # Simulate SNR improvement

    # 🔹 Re-evaluate after action
    input_seq_new = np.expand_dims(new_window, axis=0)
    new_probs = model.predict(input_seq_new, verbose=0)
    new_pred = class_order[np.argmax(new_probs)]
    outcome = "Success" if new_pred == "Good" else "No Improvement"

    print(f"📝 Logged action: {action} → Outcome: {outcome}")
    add_to_memory(window, action, outcome)


Class Order: ['Good', 'Moderate', 'Poor']
Shape of X_seq: (14986, 15, 7)
Shape of y_seq: (14986,)

[Step 1] True Label: Good
Predicted Probabilities: [[9.4825530e-01 5.1550705e-02 1.9394481e-04]]
Predicted State: Good
✅ Window classified as GOOD → No action needed.

[Step 2] True Label: Moderate
Predicted Probabilities: [[2.8539702e-04 9.9943703e-01 2.7749376e-04]]
Predicted State: Moderate
ℹ️ No similar case in memory → Trying actions from scratch.
📝 Logged action: Lower Modulation to QPSK + Max FEC → Outcome: No Improvement

[Step 3] True Label: Good
Predicted Probabilities: [[9.9970645e-01 2.8618981e-04 7.3974179e-06]]
Predicted State: Good
✅ Window classified as GOOD → No action needed.

[Step 4] True Label: Moderate
Predicted Probabilities: [[3.3827734e-04 9.9936920e-01 2.9256818e-04]]
Predicted State: Moderate
🔍 Found similar case (sim=0.04) → Previous Outcome: No Improvement
📝 Logged action: Lower Modulation to QPSK + Max FEC → Outcome: No Improvement

[Step 5] True Label: Poor
